# Limpieza de datos

In [36]:
import pandas as pd
df = pd.read_csv('BaseDefinitivaINDICES-2005-2024.csv', low_memory=False)

## Redefinición de nombres de columnas
Como la base de datos tiene nombres de columnas con carácteres especiales, se realiza una estandarización de los nombres de columnas, reemplazando espacios, tildes y otros carácteres especiales que se puedan encontrar.
Esto se hace para evitar problemas futuros en el llamado de las columnas y para facilitar el trabajo del dataset.

In [37]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('á', 'a')
df.columns = df.columns.str.replace('é', 'e')
df.columns = df.columns.str.replace('í', 'i')
df.columns = df.columns.str.replace('ó', 'o')
df.columns = df.columns.str.replace('ú', 'u')
df.columns = df.columns.str.replace('ñ', 'ni')
df.columns = df.columns.str.replace('/', '')
df.columns = df.columns.str.replace('º', '')
df.columns = df.columns.str.replace('(', '', regex=False)
df.columns = df.columns.str.replace(')', '', regex=False)
df.columns = df.columns.str.replace('.', '', regex=False)
df.columns = df.columns.str.lower()
print(df.columns)

Index(['anio', 'cod_institucion', 'nombre_institucion', 'tipo_institucion',
       'clasificacion1', 'clasificacion2', 'clasificacion3', 'clasificacion4',
       'clasificacion5', 'clasificacion6', 'nombre_de_la_sede',
       'comuna_donde_se_imparte_la_carrera_o_programa', 'nombre_region',
       'orden_geografico_de_la_region_norte_asur', 'cod_carrera',
       'carrera_generica', 'nombre_programa', 'mencion_o_especialidad',
       'horario', 'tipo_programa', 'area_conocimiento', 'idgenerocarrera',
       'tipo_carrera', 'ingresodirecto', 'anio_inicio_actividades',
       'nombre_del_campus', 'duracion_en_semestres', 'cod_campus', 'cod_sede',
       'titulo', 'grado_academico',
       'maximo_puntaje_promedio_matematicas_y_lenguaje',
       'promedio_puntaje_promedio_matematicas_y_lenguaje',
       'minimo_puntaje_promedio_matematicas_y_lenguaje',
       'puntaje_de_corte_primer_seleccionado',
       'puntaje_de_corte_promedio_de_la_carrera',
       'puntaje_de_corte_ultimo_selecciona

## Valores duplicados
Como se pudo observar en la fase de análisis exploratorio, en la base de datos se justifican los registros duplicados debido a que una misma carrera puede ser impartida múltiples añospor una misma institución además de ser impartida por múltiples instituciones. Por lo mismo, para determinar la existencia deregistros duplicados se realiza en base a los campos "Año", "Cód. Institución" y "Cód. Carrera". En este sentido, no se encuentran registros duplicados en la base de datos.

## Valores nulos
Como primera medida, se considera como regla dejar fuera las columnas que tienen más de un 50% de datos nulos debido a la falta de información suficiente. 

Para el caso de la columna "codgo_sies" que tiene un 37.4% de registros nulos, también se deja fuera porque al ser un código identificador de casos específicos, una imputación no tiene sentido.

....................




In [38]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
cols_with_more_than50_mv = missing_values[missing_values > 50]
print(missing_values)

minimo_puntaje_ranking                              91.013109
maximo_puntaje_ranking                              91.007523
promedio_puntaje_ranking                            90.946081
minimo_puntaje_nem                                  90.567975
maximo_puntaje_nem                                  90.562390
promedio_puntaje_nem                                90.482901
mencion_o_especialidad                              86.162611
puntaje_de_corte_promedio_de_la_carrera             84.557380
puntaje_de_corte_primer_seleccionado                84.516132
puntaje_de_corte_ultimo_seleccionado                83.641762
minimo_puntaje_promedio_matematicas_y_lenguaje      82.135783
promedio_puntaje_promedio_matematicas_y_lenguaje    82.067896
maximo_puntaje_promedio_matematicas_y_lenguaje      81.942433
n_alumnos_ingreso_via_psu_o_pdt                     81.325433
n_alumnos_ingreso_otra_via                          66.718943
grado_academico                                     62.602314
nombre_d

In [39]:
# Se descartan las columnas con más del 50% de valores faltantes
df = df.drop(cols_with_more_than50_mv.index, axis=1)
# Se descartan las columnas que no aportan información relevante
df = df.drop(columns=['codgo_sies', 'valor_de_arancel', 'valor_de_matricula', 'valor_del_titulo', 'titulo'], axis=1)

In [40]:
# Se imputan los valores faltantes de las variables numéricas con la media redondeada de la carrera genérica 
df['matricula_primer_anio_extranjeros'] = df.groupby('carrera_generica')['matricula_primer_anio_extranjeros'].transform(lambda x: x.fillna(x.mean()))
df['matricula_total_extranjeros'] = df.groupby('carrera_generica')['matricula_total_extranjeros'].transform(lambda x: x.fillna(x.mean()))
df['matricula_primer_anio_mujeres'] = df.groupby('carrera_generica')['matricula_primer_anio_mujeres'].transform(lambda x: x.fillna(x.mean()))
df['matricula_primer_anio_hombres'] = df.groupby('carrera_generica')['matricula_primer_anio_hombres'].transform(lambda x: x.fillna(x.mean()))
df['matricula_total_mujeres'] = df.groupby('carrera_generica')['matricula_total_mujeres'].transform(lambda x: x.fillna(x.mean()))
df['matricula_total_hombres'] = df.groupby('carrera_generica')['matricula_total_hombres'].transform(lambda x: x.fillna(x.mean()))
df['duracion_en_semestres'] = df.groupby('carrera_generica')['duracion_en_semestres'].transform(lambda x: x.fillna(x.mean()))

# Se redondean los valores de las variables numéricas
df['matricula_primer_anio_extranjeros'] = df['matricula_primer_anio_extranjeros'].round()
df['matricula_total_extranjeros'] = df['matricula_total_extranjeros'].round()
df['matricula_primer_anio_mujeres'] = df['matricula_primer_anio_mujeres'].round()
df['matricula_primer_anio_hombres'] = df['matricula_primer_anio_hombres'].round()
df['matricula_total_mujeres'] = df['matricula_total_mujeres'].round()
df['matricula_total_hombres'] = df['matricula_total_hombres'].round()
df['duracion_en_semestres'] = df['duracion_en_semestres'].round()

# Se imputan los valores faltantes de vacantes con la suma de matricula_primer_anio_mujeres, matricula_primer_anio_hombres y matricula_primer_anio_extranjeros
df['vacantes'] = df['vacantes'].fillna(df['matricula_primer_anio_mujeres'] + df['matricula_primer_anio_hombres'] + df['matricula_primer_anio_extranjeros'])

# Se eliminan las filas con valores faltantes en las variables carrera_generica, matricula_primer_anio_mujeres y matricula_primer_anio_hombres
df = df.dropna(subset=['carrera_generica', 'matricula_primer_anio_mujeres', 'matricula_primer_anio_hombres'])


In [41]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
print(missing_values)

Series([], dtype: float64)


# Creación de nuevas variables
Como la suma de matricula_total_mujeres y matricula_total_hombres no es igual a matricula_total, se crea una nueva variable matricula_total_nacional

También se crean variables que representan el porcentaje de matrícula total de hombres y mujeres

In [46]:
print(f'''
      Matrícula total hombres: {df["matricula_total_hombres"].sum()}, 
      Matrícula total mujeres: {df["matricula_total_mujeres"].sum()},
      Suma de matrícula total hombres y mujeres: {df["matricula_total_hombres"].sum() + df["matricula_total_mujeres"].sum()}, 
      Matrícula total: {df["matricula_total"].sum()}
''')
df['matricula_total_nacional'] = df['matricula_total_mujeres'] + df['matricula_total_hombres']


      Matrícula total hombres: 10549917.0, 
      Matrícula total mujeres: 11338212.0,
      Suma de matrícula total hombres y mujeres: 21888129.0, 
      Matrícula total: 21768589



In [47]:
df['percent_matricula_total_hombres'] = df['matricula_total_hombres'] / df['matricula_total_nacional']
df['percent_matricula_total_mujeres'] = df['matricula_total_mujeres'] / df['matricula_total_nacional']

In [48]:
df.to_csv('BaseDefinitivaINDICES-2005-2024_clean.csv', index=False)